In [68]:
from PIL import Image, ImageFilter
from google.cloud import vision
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import glob
import base64
import io
import os

#Funciones generales para uso global
import numpy as np
import time
from shapely.geometry import Polygon, Point
import geopandas as gpd
from shapely.ops import cascaded_union
import rasterio
from rasterio.mask import mask
from shapely import geometry
import pandas as pd
from geotiff import GeoTiff


#Funciones empleadas en la detección de texto
import os, sys
import numpy as np
import cv2
import time
from imutils.object_detection import non_max_suppression

import rasterio
from rasterio.enums import Resampling

# Model v1: Split the image, detect text areas, and convert to text strings

## Splitting the image

In [69]:
# Takes a Rasterio dataset and splits it into squares of dimensions squareDim * squareDim
def splitImageIntoCells(img, filename, num_imgs=3):
    squareDim_wide = img.shape[1] // num_imgs
    squareDim_height = img.shape[0] // num_imgs
    
    numberOfCellsWide = img.shape[1] // squareDim_wide
    numberOfCellsHigh = img.shape[0] // squareDim_height
    x, y = 0, 0
    count = 0
    for hc in range(numberOfCellsHigh):
        y = hc * squareDim_height
        for wc in range(numberOfCellsWide):
            x = wc * squareDim_wide
            geom = getTileGeom(img.transform, x, y, squareDim_wide, squareDim_height)
            getCellFromGeom(img, geom, filename, count)
            count = count + 1

# Generate a bounding box from the pixel-wise coordinates using the original datasets transform property
def getTileGeom(transform, x, y, squareDim_wide, squareDim_height):
    corner1 = (x, y) * transform
    corner2 = (x + squareDim_wide, y + squareDim_height) * transform
    return geometry.box(corner1[0], corner1[1],
                        corner2[0], corner2[1])

# Crop the dataset using the generated box and write it out as a GeoTIFF
def getCellFromGeom(img, geom, filename, count):
    crop, cropTransform = mask(img, [geom], crop=True)
    writeImageAsGeoTIFF(crop,
                        cropTransform,
                        img.meta,
                        img.crs,
                        filename+"_"+str(count))

# Write the passed in dataset as a GeoTIFF
def writeImageAsGeoTIFF(img, transform, metadata, crs, filename):
    metadata.update({"driver":"GTiff",
                     "height":img.shape[1],
                     "width":img.shape[2],
                     "transform": transform,
                     "crs": 'EPSG:4686'})
    with rasterio.open(filename+".tif", "w", **metadata) as dest:
        dest.write(img)

In [70]:
def detect_text(bytes_img):
    """
    Uses the Google Vision API to extract text from
    an image.
    
    Arguments
    ---------
    file_path: str
               path of the image to process.
    
    Outputs
    -------
    response: AnnotateImageResponse object
              json like format with bounding box and other
              relevant information.
    text: str
          text extracted from the image.
    """
    client = vision.ImageAnnotatorClient()    
    image = vision.Image(content=bytes_img)
    response = client.document_text_detection(image=image)
    text = response.full_text_annotation.text
    
    return response, text


def getbytesimg_from_path(filepath):
    """
    Obtains the bytes base64 format of an image from a 
    local file path.
    
    Arguments
    ---------
    filepath: str
              Path of the image file to convert
    
    Output
    ------
    bytes_img: bytes
               base64 format of the image
    """
    with open(filepath, "rb") as image_file:
        bytes_img = image_file.read()
    
    return bytes_img

In [71]:
def split_images(filepath, num_imgs=3):
    """
    Split a large image into a grid of 3x3
    smaller images.
    
    Arguments:
    ---------
    filepath: str
              file path of the large image
    num_imgs: int (optional)
              Number of rows and columns of the grid
              
    Output
    ------
    None
    
    """
    img = rasterio.open(filepath)
    splitImageIntoCells(img, "split_out/output_data")

In [72]:
def geotif_to_jpeg(tif_filename):
    """
    Converts geotif image from disk and saves it 
    in the same folder in jpeg format.
    
    Arguments
    ---------
    tif_filename: str
                  path of the tif file to convert
    
    Output
    ------
    None
    
    """
    with rasterio.open(tif_filename) as infile:    
        profile = infile.profile    
        profile['driver']='JPEG'
        jpeg_filename = tif_filename.replace(".tif", ".jpeg")
        with rasterio.open(jpeg_filename, 'w', **profile) as dst:
            dst.write(infile.read())

In [73]:
def get_text_coords(response):
    """
    Extracts the geographic names and bounding boxes coordinates
    from an AnnotateImageResponse object (Google).
    
    Arguments
    ---------
    response: AnnotateImageResponse (Google Vision API)
              Response object after calling the document_text_detection
              function
              
    Outputs
    -------
    palabras_google: list
                     List of strings containing the geographic names
    boundings_google: list
                      List of lists of 4 vertices that define the bounding 
                      box of each geographic name
    confidence_google: list
                       List of floats representing the confidence of the 
                       Google API at detecting each handwritten text
    """
    
    palabras_google = []
    boundings_google = []
    confidence_google = []
    
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            palabra_google = ''
            boundings_google.append(block.bounding_box)
            confidence_google.append(block.confidence)
            for parrafos in block.paragraphs:
                for palabras in parrafos.words:
                    for simbolo in palabras.symbols:
                        palabra_google = palabra_google+simbolo.text
                    palabra_google = palabra_google+' '
            palabras_google.append(palabra_google.rstrip())
            
    return palabras_google, boundings_google, confidence_google

In [74]:
def get_coords(boundings, word_index, point_index, axis):
    """
    Returns x or y coordinate from the list of vertices.
    
    Arguments
    ---------
    boundings: list
               List of bounding boxes with the vertices of 
               each bounding box
    word_index: int
                Index of the desired word starting at 0
    point_index: int
                 Index of the desired point (from 0 to 3)
    coord: str
           Either x or y
           
    Output
    ------
    coordinate: float
                Coordinate of the specified word, point, and axis
    
    """
    if axis == "x":
        coordinate = boundings[word_index].vertices[point_index].x
    else:
        coordinate = boundings[word_index].vertices[point_index].y
    
    return coordinate

In [75]:
def create_geometries(filepath):
    """
    Splits the image in a grid, uses Google Vision API to extract text
    and the bounding boxes. This method creates geojson files with 
    the geographic names, their respective geodesic coordinates, the 
    confidence, and the geometries of the bounding boxes.
    
    The geojson files are saved in the folder geometries.
    
    Arguments
    ---------
    filepath: str
              Path to the geotiff image file to process
              
    Output
    ------
    None
    """

    split_images(filepath)
    images_to_proces = os.listdir(path='split_out')

    for path in images_to_proces:
        if ('.tif' in path):
            geotif_to_jpeg("split_out/" + path)
            sub_image_jpeg = path.replace('.tif','.jpeg')
            response, text = detect_text(getbytesimg_from_path("split_out/" + sub_image_jpeg))
            words, boundings, confidence = get_text_coords(response)
            img = rasterio.open("split_out/" + sub_image_jpeg)

            # Generating the sub_geometries
            geometries = []
            centroids = []

            for i in range(len(words)):
                aux_polygon = Polygon([img.xy(get_coords(boundings,i,0,"y"),get_coords(boundings,i,0,"x")),
                                       img.xy(get_coords(boundings,i,1,"y"),get_coords(boundings,i,1,"x")),
                                       img.xy(get_coords(boundings,i,2,"y"),get_coords(boundings,i,2,"x")),
                                       img.xy(get_coords(boundings,i,3,"y"),get_coords(boundings,i,3,"x")),
                                       img.xy(get_coords(boundings,i,0,"y"),get_coords(boundings,i,0,"x"))])
                geometries.append(aux_polygon)
                centroids.append(aux_polygon.representative_point())

            sub_img_gdf = gpd.GeoDataFrame(columns=["toponimo_ocr","confidence",
                "centroide_longitud","centroide_latitud","geometry"], crs=str(img.crs))
            sub_img_gdf["geometry"] = geometries
            sub_img_gdf["toponimo_ocr"] = words
            sub_img_gdf["confidence"] = confidence
            sub_img_gdf["centroide_longitud"] = [x.coords[0][0] for x in centroids]
            sub_img_gdf["centroide_latitud"] = [x.coords[0][1] for x in centroids]
            name_geometry = 'geometries/'+sub_image_jpeg.replace('.jpeg','.geojson')
            try:
                sub_img_gdf.to_file(name_geometry, driver="GeoJSON")
            except:
                print('Empty text detected, no geometries generated!')

In [90]:
def combine_geometries():
    """
    Takes all the geojson files in the geometries folder and combine
    them into a single geojson file. It combines bounding boxes that 
    intersect each other and their respective geographic names. It 
    deletes the rows whose name has only numbers.
    
    Arguments
    ---------
    None
    
    Output
    ------
    all_toponyms_gdf: GeoDataFrame
                      GeoDataFrame with all the toponyms from the original
                      image. It contains the geographic name, geodesic 
                      coordinates of the centroid, confidence and the 
                      geometry of the bounding boxes.
    
    """
    geometries_to_process = os.listdir(path='geometries')    
    rectangles = []
    all_toponyms_gdf = gpd.GeoDataFrame(columns=["toponimo_ocr","confidence",
                "centroide_longitud","centroide_latitud","geometry"],crs=str(org_img.crs))

    for geometry in geometries_to_process:
        if ".geojson" in geometry:
            file = gpd.read_file('geometries/' + geometry)
            all_toponyms_gdf = all_toponyms_gdf.append(file)

    all_toponyms_gdf.reset_index(drop=True, inplace=True)
    all_toponyms_gdf.to_file('text_detected.geojson',driver='GeoJSON')
    
    return all_toponyms_gdf

In [91]:
def get_image_corners(filepath):
    """
    Returns the geodesic coordinates of the original image.
    
    Arguments
    ---------
    filepath: str
              Path of the original image
    
    Output
    ------
    image_corners: dict
                   Dictionary with two keys: "upper_left" and "lower_left"
                   The values are tuples with the latitude and longitude
                   of the image corners.
    """
    original_img = rasterio.open(filepath)
    image_corners = {
        "upper_left": original_img.xy(0,0),
        "lower_right": original_img.xy(org_img.shape[0], org_img.shape[1])
    }
    
    return image_corners

In [92]:
def empty_folders():
    """
    Remove all temporary output files in the "split_out" and
    "geometries" folders. It also deletes the final output file
    detected_text.geojson.
    
    Arguments
    ---------
    None
    
    Output
    ------
    None
    """
    folders = ["split_out", "geometries"]
    
    for folder in folders:
        files = glob.glob(f"{folder}/*")
        for f in files:
            os.remove(f)
    try:
        os.remove("text_detected.geojson")
    except:
        print("text_detected.geojson already deleted!")

In [93]:
aerophotos = glob.glob("geotiffs/*.tif")
aerophotos

['geotiffs/M-1390 F-42286.tif',
 'geotiffs/M-1390 F-42290.tif',
 'geotiffs/C-1974 F-238.tif',
 'geotiffs/C-2070 F-252.tif',
 'geotiffs/C-2070 F-250.tif',
 'geotiffs/C-1974 F-240.tif']

## Suggested procedure

1. Delete all the temporary files (`empty_folders()`)
2. Create individual geometries from the path of the original image (`create_geometries(path)`)
3. Combine geometries into single geojson file (`combine_geometries()`)
4. Get coordinates of the corners of the original image (`get_image_corners(path)`)

### Testing for each of the 6 geotiff images

In [94]:
# M-1390 F-42286
empty_folders()
create_geometries(aerophotos[0])
all_toponyms_img1 = combine_geometries()
get_image_corners(aerophotos[0])

<ipython-input-69-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-69-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


Empty text detected, no geometries generated!
Empty text detected, no geometries generated!


{'upper_left': (-75.81961718134531, 3.9789168373452815),
 'lower_right': (-75.70965295164946, 3.8677789335113406)}

In [95]:
# M-1390 F-42290
empty_folders()
create_geometries(aerophotos[1])
all_toponyms_img2 = combine_geometries()
get_image_corners(aerophotos[1])

<ipython-input-69-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-69-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


{'upper_left': (-75.79279845128181, 4.112223605281884),
 'lower_right': (-75.69306024028181, 4.011420865281885)}

In [98]:
# C-1974 F-238
%%timeit
empty_folders()
create_geometries(aerophotos[2])
all_toponyms_img3 = combine_geometries()
get_image_corners(aerophotos[2])

CPU times: user 6 µs, sys: 3 µs, total: 9 µs
Wall time: 14.8 µs


<ipython-input-69-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-69-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


Empty text detected, no geometries generated!


{'upper_left': (-75.8740351937534, 3.8952368917533815),
 'lower_right': (-75.80455927230668, 3.8250194374035167)}

In [101]:
# C-1974 F-252
%%time
empty_folders()
create_geometries(aerophotos[3])
all_toponyms_img4 = combine_geometries()
get_image_corners(aerophotos[3])

<ipython-input-69-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-69-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


{'upper_left': (-75.82882085855019, 3.952130957550283),
 'lower_right': (-75.73956849910715, 3.8619259871436658)}

In [103]:
%%time
# C-1974 F-250
empty_folders()
create_geometries(aerophotos[4])
all_toponyms_img5 = combine_geometries()
get_image_corners(aerophotos[4])

<ipython-input-69-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-69-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


CPU times: user 2.09 s, sys: 153 ms, total: 2.24 s
Wall time: 6.13 s


{'upper_left': (-75.82953946582832, 4.001018766828459),
 'lower_right': (-75.73684882941001, 3.9073388221590064)}

In [104]:
%%time
# C-1974 F-240
empty_folders()
create_geometries(aerophotos[5])
all_toponyms_img6 = combine_geometries()
get_image_corners(aerophotos[5])

<ipython-input-69-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-69-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


CPU times: user 2.03 s, sys: 163 ms, total: 2.19 s
Wall time: 5.63 s


{'upper_left': (-75.87331615888398, 3.9374634048839634),
 'lower_right': (-75.80694433765417, 3.8703831815202085)}

In [105]:
%%time
# C-1974 F-240
empty_folders()
create_geometries("282IID_1983.tif")
all_toponyms_sheet1 = combine_geometries()
get_image_corners("282IID_1983.tif")

<ipython-input-69-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-69-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


CPU times: user 5.22 s, sys: 607 ms, total: 5.83 s
Wall time: 15.8 s


{'upper_left': (-75.29669677534784, 3.789153219486796),
 'lower_right': (-75.2503365180129, 3.741840900471164)}

In [107]:
all_toponyms_sheet1.head(50)

,toponimo_ocr,confidence,centroide_longitud,centroide_latitud,geometry
0,2009 .,0.92,-75.242332,3.703368,"POLYGON ((-75.24120 3.70419, -75.24225 3.70195..."
1,800,0.97,-75.225253,3.703167,"POLYGON ((-75.22561 3.70270, -75.22546 3.70371..."
2,I,0.24,-75.209657,3.702619,"POLYGON ((-75.21007 3.70331, -75.20965 3.70343..."
3,Queb :,0.66,-75.202124,3.703234,"POLYGON ((-75.20263 3.70435, -75.20110 3.70256..."
4,Atienda Bombaza,0.73,-75.243009,3.699640,"POLYGON ((-75.24579 3.69999, -75.24023 3.69999..."
5,Chond,0.78,-75.209153,3.698439,"POLYGON ((-75.21001 3.69956, -75.20780 3.69806..."
6,1/614,0.84,-75.244037,3.698266,"POLYGON ((-75.24480 3.69868, -75.24320 3.69845..."
7,RANDE,0.84,-75.247815,3.696402,"POLYGON ((-75.24918 3.69670, -75.24645 3.69672..."
8,-600,0.96,-75.216803,3.695153,"POLYGON ((-75.21707 3.69455, -75.21709 3.69576..."
9,008,0.90,-75.223350,3.692607,"POLYGON ((-75.22367 3.69326, -75.22267 3.69240..."
